In [306]:
import pandas as pd
from tqdm import tqdm
import re
import pickle
import csv
import pandas as pd
from pandas import DataFrame 
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Kkma
from konlpy.tag import Okt
from collections import Counter
from konlpy.tag import Mecab
import json


In [3]:
file_names = ['Watcha_Contents','Naver_Contents','Tistory_Contents']

In [429]:
def make_topic_dict(file_name,file_total_num):
    movie = pd.read_csv('./total_review.csv',index_col=0,header=0)
    movie.replace('[]',np.nan,inplace=True)
    movie_info = tuple(zip(movie.index,movie.nfid))
    movie_dict = {f'{items[0]}_{items[1]}' : {} for items in movie_info}

    total_topic_df = []
    for file in file_name:
        for i in range(1,file_total_num+1):
            df = pd.read_csv(f'./{file}_topic{i}.csv',encoding='utf-8',index_col=0)
            total_topic_df.append(df)

    for num in range(len(total_topic_df)):
        for i,(k,v) in tqdm(enumerate(movie_info)):
            keyword=total_topic_df[num][f'{k}_{v}_keyword'].dropna()
            weight=total_topic_df[num][f'{k}_{v}_weight'].dropna()
            
            try:
                weight=weight[1:].astype(float)
                keyword_dic = { name:value for name, value in zip(keyword[1:], weight) }
            except:
                continue
            
            if num==0:
                movie_dict[f'{k}_{v}'] =  keyword_dic
            else:
                intersection_keyword = list(set(movie_dict[f'{k}_{v}'].keys())&set(keyword_dic.keys()))
                subtraction_keyword = list(set(keyword_dic.keys())-set(movie_dict[f'{k}_{v}'].keys()))
                for i_k in intersection_keyword:
                    movie_dict[f'{k}_{v}'][i_k]=\
                        float(movie_dict[f'{k}_{v}'][i_k])+\
                            float(keyword_dic[i_k])
                for s_k in subtraction_keyword:
                    movie_dict[f'{k}_{v}'][s_k]=float(keyword_dic[s_k])
    return movie_dict

In [388]:
def top_n(count_dict, reverse, n=3):
    return dict(sorted(count_dict.items(), reverse=reverse, key=lambda x: x[1])[:n])

In [430]:
movie_json = make_topic_dict(file_names,3)

1437it [00:00, 3009.22it/s]
1437it [00:00, 2331.08it/s]
1437it [00:00, 2614.40it/s]
1437it [00:00, 2274.46it/s]
1437it [00:00, 2458.62it/s]
1437it [00:00, 2343.14it/s]
1437it [00:00, 3273.55it/s]
1437it [00:00, 3105.63it/s]
1437it [00:00, 2855.67it/s]


In [431]:
with open(f'./movie_topic2.json','w') as f:
    json.dump(movie_json,
            f,ensure_ascii=False,indent=4)

In [432]:
with open(f'./movie_topic2.json', 'r') as f:
    json_data = json.load(f)

In [441]:
top_n(json_data['시그널_80987077'],True,150)

{'시즌': 0.4601148,
 '재밌': 0.40517059,
 '영화': 0.22664501,
 '이재한': 0.17317721,
 '형사': 0.16158855,
 '시그널': 0.15908729,
 '김은희': 0.11179001,
 '시작': 0.106639035,
 '기대': 0.09148259,
 '사랑': 0.087158632,
 '타임': 0.081503704,
 '작품': 0.07972412,
 '장면': 0.079018643,
 '주행': 0.07100214,
 '행복': 0.069647401,
 '느낌': 0.068425864,
 '마지막': 0.06799764,
 '프리': 0.06399858700000001,
 '박해영': 0.061357014,
 '대사': 0.059616956,
 '명작': 0.05831014,
 '박수': 0.0513755635,
 '간절': 0.046827529,
 '세상': 0.045707166,
 '이야기': 0.044268698,
 '훌륭': 0.044097003,
 '본방': 0.043215129000000005,
 '범죄': 0.04280265,
 '무전': 0.041338779000000006,
 '한편': 0.03778506,
 '평가': 0.036274666,
 '감독': 0.034989327,
 '언니': 0.034259928,
 '웰메이드': 0.03395032,
 '경위': 0.03374224,
 '명품': 0.033709268,
 '소취': 0.033662092,
 '매력': 0.03336074,
 '각본': 0.032785393,
 '아깝': 0.032479166000000004,
 '취향': 0.032388373,
 '스릴': 0.031745851,
 '후회': 0.031364756,
 '재미': 0.0311613,
 '정의': 0.030474007,
 '공존': 0.030243208,
 '슬립': 0.029027356,
 '감사': 0.02878574,
 '라인': 0.02831123